En este notebook vamos a visualizar todos los datos, y definir qué forma tendrá el dataset mergeado final.

In [5]:
import pandas as pd

def load_data(file_path):
    """Load data from a CSV file into a pandas DataFrame."""
    return pd.read_csv(file_path)

alumnnos21 = pd.read_csv('2021-alumnos.csv', sep=';', encoding='latin-1')
alumnnos22 = pd.read_csv('2022-alumnos.csv', sep=';', encoding='latin-1')
alumnnos23 = pd.read_csv('2023-alumnos.csv', sep=';', encoding='latin-1')
alumnnos24 = pd.read_csv('2024-alumnos.csv', sep=';', encoding='latin-1')
profesores21 = pd.read_csv('2021-profesores.csv', sep=';', encoding='latin-1')
profesores22 = pd.read_csv('2022-profesores.csv', sep=';', encoding='latin-1')
profesores23 = pd.read_csv('2023-profesores.csv', sep=';', encoding='latin-1')
profesores24 = pd.read_csv('2024-profesores.csv', sep=';', encoding='latin-1')

alumnnos21.head()

,CURSO,COD_RUCT,TITULACION,CENTRO,SATIS_ALU_GESTION_TIT
0,2020-21,2500739,GRADO EN ARQUITECTURA TÉCNICA,E.T.S. DE INGENIERÍA DE EDIFICACIÓN,6.59
1,2020-21,2500801,GRADO EN CIENCIAS AMBIENTALES,ESCUELA POLITECNICA SUPERIOR DE GANDIA,7.13
2,2020-21,2501354,GRADO EN COMUNICACIÓN AUDIOVISUAL,ESCUELA POLITECNICA SUPERIOR DE GANDIA,6.25
3,2020-21,2501355,GRADO EN TURISMO,ESCUELA POLITECNICA SUPERIOR DE GANDIA,6.43
4,2020-21,2501356,GRADO EN INGENIERÍA EN DISEÑO INDUSTRIAL Y DES...,E. POLITÉCNICA SUPERIOR DE ALCOY,6.80


In [6]:
# Explorar la estructura de todos los datasets
print("=== ESTRUCTURA DE DATOS DE ALUMNOS ===")
datasets_alumnos = [
    ("2021", alumnnos21),
    ("2022", alumnnos22), 
    ("2023", alumnnos23),
    ("2024", alumnnos24)
]

for year, df in datasets_alumnos:
    print(f"\n--- Alumnos {year} ---")
    print(f"Shape: {df.shape}")
    print(f"Columnas: {list(df.columns)}")
    print(f"Sample:")
    print(df.head(2))

print("\n" + "="*60)
print("=== ESTRUCTURA DE DATOS DE PROFESORES ===")
datasets_profesores = [
    ("2021", profesores21),
    ("2022", profesores22),
    ("2023", profesores23), 
    ("2024", profesores24)
]

for year, df in datasets_profesores:
    print(f"\n--- Profesores {year} ---")
    print(f"Shape: {df.shape}")
    print(f"Columnas: {list(df.columns)}")
    print(f"Sample:")
    print(df.head(2))

=== ESTRUCTURA DE DATOS DE ALUMNOS ===

--- Alumnos 2021 ---
Shape: (113, 5)
Columnas: ['CURSO', 'COD_RUCT', 'TITULACION', 'CENTRO', 'SATIS_ALU_GESTION_TIT']
Sample:
     CURSO  COD_RUCT                     TITULACION  \
0  2020-21   2500739  GRADO EN ARQUITECTURA TÉCNICA   
1  2020-21   2500801  GRADO EN CIENCIAS AMBIENTALES   

                                   CENTRO  SATIS_ALU_GESTION_TIT  
0     E.T.S. DE INGENIERÍA DE EDIFICACIÓN                   6.59  
1  ESCUELA POLITECNICA SUPERIOR DE GANDIA                   7.13  

--- Alumnos 2022 ---
Shape: (116, 5)
Columnas: ['CURSO', 'COD_RUCT', 'TITULACION', 'CENTRO', 'SATIS_ALU_GESTION_TIT']
Sample:
     CURSO  COD_RUCT                     TITULACION  \
0  2021-22   2500739  GRADO EN ARQUITECTURA TÉCNICA   
1  2021-22   2500801  GRADO EN CIENCIAS AMBIENTALES   

                                   CENTRO SATIS_ALU_GESTION_TIT  
0     E.T.S. DE INGENIERÍA DE EDIFICACIÓN              6,750000  
1  ESCUELA POLITECNICA SUPERIOR DE GANDIA 

In [7]:
# Análisis específico de columnas
print("=== ANÁLISIS DE COLUMNAS ===")

# Verificar si todas las tablas de alumnos tienen las mismas columnas
alumnos_columns = [set(df.columns) for _, df in datasets_alumnos]
print("¿Todas las tablas de alumnos tienen las mismas columnas?", len(set(frozenset(s) for s in alumnos_columns)) == 1)

# Verificar si todas las tablas de profesores tienen las mismas columnas  
profesores_columns = [set(df.columns) for _, df in datasets_profesores]
print("¿Todas las tablas de profesores tienen las mismas columnas?", len(set(frozenset(s) for s in profesores_columns)) == 1)

print(f"\nColumnas alumnos: {list(alumnnos21.columns)}")
print(f"Columnas profesores: {list(profesores21.columns)}")

# Columnas comunes entre alumnos y profesores
common_cols = set(alumnnos21.columns) & set(profesores21.columns)
print(f"\nColumnas comunes: {list(common_cols)}")

# Columnas únicas de cada grupo
alumnos_unique = set(alumnnos21.columns) - set(profesores21.columns)
profesores_unique = set(profesores21.columns) - set(alumnnos21.columns)
print(f"Columnas únicas alumnos: {list(alumnos_unique)}")
print(f"Columnas únicas profesores: {list(profesores_unique)}")

=== ANÁLISIS DE COLUMNAS ===
¿Todas las tablas de alumnos tienen las mismas columnas? True
¿Todas las tablas de profesores tienen las mismas columnas? True

Columnas alumnos: ['CURSO', 'COD_RUCT', 'TITULACION', 'CENTRO', 'SATIS_ALU_GESTION_TIT']
Columnas profesores: ['CURSO', 'COD_RUCT', 'TITULACION', 'CENTRO', 'SATIS_PROF_GESTION_TIT']

Columnas comunes: ['CENTRO', 'CURSO', 'COD_RUCT', 'TITULACION']
Columnas únicas alumnos: ['SATIS_ALU_GESTION_TIT']
Columnas únicas profesores: ['SATIS_PROF_GESTION_TIT']


## Creación del Panel de Datos Unificado

Ahora vamos a crear un dataset unificado que contenga:
- Todas las carreras/masters de todos los años
- Las puntuaciones de satisfacción tanto de alumnos como de profesores
- Una estructura de panel de datos con año, carrera, y ambas puntuaciones

In [8]:
# Paso 1: Concatenar todos los datos de alumnos
alumnos_all = pd.concat([
    alumnnos21, alumnnos22, alumnnos23, alumnnos24
], ignore_index=True)

# Paso 2: Concatenar todos los datos de profesores  
profesores_all = pd.concat([
    profesores21, profesores22, profesores23, profesores24
], ignore_index=True)

print("Dataset alumnos unificado:")
print(f"Shape: {alumnos_all.shape}")
print(f"Años únicos: {sorted(alumnos_all['CURSO'].unique())}")

print("\nDataset profesores unificado:")
print(f"Shape: {profesores_all.shape}")
print(f"Años únicos: {sorted(profesores_all['CURSO'].unique())}")

Dataset alumnos unificado:
Shape: (475, 5)
Años únicos: ['2020-21', '2021-22', '2022-23', '2023-24']

Dataset profesores unificado:
Shape: (480, 5)
Años únicos: ['2020-21', '2021-22', '2022-23', '2023-24']


In [9]:
# Paso 3: Mergear alumnos y profesores por las columnas comunes
panel_satisfaccion = pd.merge(
    alumnos_all, 
    profesores_all, 
    on=['CURSO', 'COD_RUCT', 'TITULACION', 'CENTRO'],
    how='outer',  # outer join para incluir todas las carreras de ambas fuentes
    suffixes=('_alumnos', '_profesores')
)

print("Panel de datos unificado:")
print(f"Shape: {panel_satisfaccion.shape}")
print(f"Columnas: {list(panel_satisfaccion.columns)}")
print("\nPrimeras filas:")
print(panel_satisfaccion.head())

Panel de datos unificado:
Shape: (483, 6)
Columnas: ['CURSO', 'COD_RUCT', 'TITULACION', 'CENTRO', 'SATIS_ALU_GESTION_TIT', 'SATIS_PROF_GESTION_TIT']

Primeras filas:
     CURSO  COD_RUCT                                         TITULACION  \
0  2020-21   2500739                      GRADO EN ARQUITECTURA TÉCNICA   
1  2020-21   2500801                      GRADO EN CIENCIAS AMBIENTALES   
2  2020-21   2501354                  GRADO EN COMUNICACIÓN AUDIOVISUAL   
3  2020-21   2501355                                   GRADO EN TURISMO   
4  2020-21   2501356  GRADO EN INGENIERÍA EN DISEÑO INDUSTRIAL Y DES...   

                                   CENTRO SATIS_ALU_GESTION_TIT  \
0     E.T.S. DE INGENIERÍA DE EDIFICACIÓN                  6.59   
1  ESCUELA POLITECNICA SUPERIOR DE GANDIA                  7.13   
2  ESCUELA POLITECNICA SUPERIOR DE GANDIA                  6.25   
3  ESCUELA POLITECNICA SUPERIOR DE GANDIA                  6.43   
4        E. POLITÉCNICA SUPERIOR DE ALCOY       

In [10]:
# Paso 4: Mejorar el dataset con columnas adicionales
# Renombrar columnas para mayor claridad
panel_satisfaccion = panel_satisfaccion.rename(columns={
    'SATIS_ALU_GESTION_TIT': 'satisfaccion_alumnos',
    'SATIS_PROF_GESTION_TIT': 'satisfaccion_profesores'
})

# Agregar año numérico para facilitar análisis
panel_satisfaccion['año'] = panel_satisfaccion['CURSO'].str[:4].astype(int)

# Agregar diferencia entre satisfacción de profesores y alumnos
panel_satisfaccion['diferencia_satis'] = (
    panel_satisfaccion['satisfaccion_profesores'] - 
    panel_satisfaccion['satisfaccion_alumnos']
)

# Agregar promedio de satisfacción
panel_satisfaccion['satisfaccion_promedio'] = (
    panel_satisfaccion['satisfaccion_alumnos'] + 
    panel_satisfaccion['satisfaccion_profesores']
) / 2

print("Dataset final mejorado:")
print(f"Shape: {panel_satisfaccion.shape}")
print(f"Columnas: {list(panel_satisfaccion.columns)}")
print("\nPrimeras filas con nuevas columnas:")
print(panel_satisfaccion[['CURSO', 'TITULACION', 'satisfaccion_alumnos', 
                         'satisfaccion_profesores', 'diferencia_satis', 
                         'satisfaccion_promedio', 'año']].head())

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [11]:
# Investigar los tipos de datos y valores problemáticos
print("Información sobre tipos de datos:")
print(panel_satisfaccion.dtypes)

print("\nValores únicos en satisfaccion_alumnos:")
print(panel_satisfaccion['satisfaccion_alumnos'].value_counts())

print("\nValores únicos en satisfaccion_profesores:")
print(panel_satisfaccion['satisfaccion_profesores'].value_counts())

print("\nValores no numéricos en satisfaccion_alumnos:")
non_numeric_alumnos = panel_satisfaccion[
    ~panel_satisfaccion['satisfaccion_alumnos'].astype(str).str.replace('.', '').str.replace(',', '').str.isdigit()
]['satisfaccion_alumnos'].unique()
print(non_numeric_alumnos)

print("\nValores no numéricos en satisfaccion_profesores:")
non_numeric_profesores = panel_satisfaccion[
    ~panel_satisfaccion['satisfaccion_profesores'].astype(str).str.replace('.', '').str.replace(',', '').str.isdigit()
]['satisfaccion_profesores'].unique()
print(non_numeric_profesores)

Información sobre tipos de datos:
CURSO                      object
COD_RUCT                    int64
TITULACION                 object
CENTRO                     object
satisfaccion_alumnos       object
satisfaccion_profesores    object
año                         int64
dtype: object

Valores únicos en satisfaccion_alumnos:
satisfaccion_alumnos
7,500000     23
6,250000     13
5,000000     11
10,000000    11
7,190000      7
             ..
4,060000      1
8,570000      1
7,400000      1
4,250000      1
3,540000      1
Name: count, Length: 271, dtype: int64

Valores únicos en satisfaccion_profesores:
satisfaccion_profesores
10,000000    23
8,750000     17
9,380000     11
9,170000     10
8,330000      8
             ..
7,930000      1
9,610000      1
9,810000      1
7,700000      1
6,790000      1
Name: count, Length: 238, dtype: int64

Valores no numéricos en satisfaccion_alumnos:
[nan]

Valores no numéricos en satisfaccion_profesores:
[nan]


In [12]:
# Corregir el formato de números (coma -> punto) y convertir a float
def fix_decimal_format(column):
    """Convierte columnas con comas decimales a float"""
    return pd.to_numeric(
        column.astype(str).str.replace(',', '.').replace('nan', None), 
        errors='coerce'
    )

# Aplicar la corrección a las columnas de satisfacción
panel_satisfaccion['satisfaccion_alumnos'] = fix_decimal_format(panel_satisfaccion['satisfaccion_alumnos'])
panel_satisfaccion['satisfaccion_profesores'] = fix_decimal_format(panel_satisfaccion['satisfaccion_profesores'])

print("Después de la corrección:")
print(panel_satisfaccion.dtypes)
print(f"\nPrimeras filas de satisfacción:")
print(panel_satisfaccion[['TITULACION', 'satisfaccion_alumnos', 'satisfaccion_profesores']].head())

print(f"\nEstadísticas de satisfaccion_alumnos:")
print(panel_satisfaccion['satisfaccion_alumnos'].describe())

print(f"\nEstadísticas de satisfaccion_profesores:")
print(panel_satisfaccion['satisfaccion_profesores'].describe())

Después de la corrección:
CURSO                       object
COD_RUCT                     int64
TITULACION                  object
CENTRO                      object
satisfaccion_alumnos       float64
satisfaccion_profesores    float64
año                          int64
dtype: object

Primeras filas de satisfacción:
                                          TITULACION  satisfaccion_alumnos  \
0                      GRADO EN ARQUITECTURA TÉCNICA                  6.59   
1                      GRADO EN CIENCIAS AMBIENTALES                  7.13   
2                  GRADO EN COMUNICACIÓN AUDIOVISUAL                  6.25   
3                                   GRADO EN TURISMO                  6.43   
4  GRADO EN INGENIERÍA EN DISEÑO INDUSTRIAL Y DES...                  6.80   

   satisfaccion_profesores  
0                     7.84  
1                     7.90  
2                     8.52  
3                     7.81  
4                     8.14  

Estadísticas de satisfaccion_alumnos:


In [13]:
# Ahora crear las columnas calculadas
# Diferencia entre satisfacción de profesores y alumnos
panel_satisfaccion['diferencia_satis'] = (
    panel_satisfaccion['satisfaccion_profesores'] - 
    panel_satisfaccion['satisfaccion_alumnos']
)

# Promedio de satisfacción
panel_satisfaccion['satisfaccion_promedio'] = (
    panel_satisfaccion['satisfaccion_alumnos'] + 
    panel_satisfaccion['satisfaccion_profesores']
) / 2

# Verificar valores faltantes
print("Valores faltantes por columna:")
print(panel_satisfaccion.isnull().sum())

print(f"\nDataset final:")
print(f"Shape: {panel_satisfaccion.shape}")
print(f"Columnas: {list(panel_satisfaccion.columns)}")

print(f"\nPrimeras filas del panel final:")
print(panel_satisfaccion[['CURSO', 'TITULACION', 'satisfaccion_alumnos', 
                         'satisfaccion_profesores', 'diferencia_satis', 
                         'satisfaccion_promedio']].head())

Valores faltantes por columna:
CURSO                       0
COD_RUCT                    0
TITULACION                  0
CENTRO                      0
satisfaccion_alumnos        8
satisfaccion_profesores     3
año                         0
diferencia_satis           11
satisfaccion_promedio      11
dtype: int64

Dataset final:
Shape: (483, 9)
Columnas: ['CURSO', 'COD_RUCT', 'TITULACION', 'CENTRO', 'satisfaccion_alumnos', 'satisfaccion_profesores', 'año', 'diferencia_satis', 'satisfaccion_promedio']

Primeras filas del panel final:
     CURSO                                         TITULACION  \
0  2020-21                      GRADO EN ARQUITECTURA TÉCNICA   
1  2020-21                      GRADO EN CIENCIAS AMBIENTALES   
2  2020-21                  GRADO EN COMUNICACIÓN AUDIOVISUAL   
3  2020-21                                   GRADO EN TURISMO   
4  2020-21  GRADO EN INGENIERÍA EN DISEÑO INDUSTRIAL Y DES...   

   satisfaccion_alumnos  satisfaccion_profesores  diferencia_satis  \
0

In [14]:
# Análisis final del panel de datos
print("=== RESUMEN DEL PANEL DE DATOS ===")
print(f"Total de observaciones: {len(panel_satisfaccion)}")
print(f"Total de titulaciones únicas: {panel_satisfaccion['TITULACION'].nunique()}")
print(f"Total de centros únicos: {panel_satisfaccion['CENTRO'].nunique()}")
print(f"Años cubiertos: {sorted(panel_satisfaccion['año'].unique())}")

print(f"\n=== ESTADÍSTICAS GENERALES ===")
print(f"Satisfacción promedio alumnos: {panel_satisfaccion['satisfaccion_alumnos'].mean():.2f}")
print(f"Satisfacción promedio profesores: {panel_satisfaccion['satisfaccion_profesores'].mean():.2f}")
print(f"Diferencia promedio (prof - alum): {panel_satisfaccion['diferencia_satis'].mean():.2f}")

print(f"\n=== COBERTURA DE DATOS ===")
coverage = panel_satisfaccion.groupby('año').agg({
    'satisfaccion_alumnos': 'count',
    'satisfaccion_profesores': 'count'
}).rename(columns={
    'satisfaccion_alumnos': 'titulaciones_con_datos_alumnos',
    'satisfaccion_profesores': 'titulaciones_con_datos_profesores'
})
print(coverage)

# Guardar el dataset final
panel_satisfaccion.to_csv('../panel_satisfaccion_UPV.csv', index=False, encoding='utf-8')
print(f"\n✅ Dataset guardado como 'panel_satisfaccion_UPV.csv'")
print(f"Ubicación: ../panel_satisfaccion_UPV.csv")

=== RESUMEN DEL PANEL DE DATOS ===
Total de observaciones: 483
Total de titulaciones únicas: 124
Total de centros únicos: 15
Años cubiertos: [np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023)]

=== ESTADÍSTICAS GENERALES ===
Satisfacción promedio alumnos: 6.92
Satisfacción promedio profesores: 8.65
Diferencia promedio (prof - alum): 1.75

=== COBERTURA DE DATOS ===
      titulaciones_con_datos_alumnos  titulaciones_con_datos_profesores
año                                                                    
2020                             113                                113
2021                             116                                116
2022                             121                                125
2023                             125                                126

✅ Dataset guardado como 'panel_satisfaccion_UPV.csv'
Ubicación: ../panel_satisfaccion_UPV.csv


## ✅ Conclusiones del Merge

### Dataset Final Creado
- **483 observaciones** combinando datos de alumnos y profesores (2020-2024)
- **124 titulaciones únicas** de **15 centros** diferentes
- **Panel de datos** con satisfacción de alumnos y profesores por año

### Estructura del Dataset Final
El archivo `panel_satisfaccion_UPV.csv` contiene:
- **CURSO**: Curso académico (2020-21, 2021-22, etc.)
- **COD_RUCT**: Código RUCT de la titulación
- **TITULACION**: Nombre de la titulación
- **CENTRO**: Centro donde se imparte
- **satisfaccion_alumnos**: Puntuación de satisfacción de alumnos (1-10)
- **satisfaccion_profesores**: Puntuación de satisfacción de profesores (1-10)
- **año**: Año numérico para análisis
- **diferencia_satis**: Diferencia entre satisfacción de profesores y alumnos
- **satisfaccion_promedio**: Promedio de ambas satisfacciones

### Hallazgos Principales
1. **Los profesores están más satisfechos** que los alumnos (8.65 vs 6.92 de media)
2. **Diferencia promedio de 1.75 puntos** entre profesores y alumnos
3. **Buena cobertura temporal**: 4 años académicos consecutivos
4. **Datos casi completos**: Solo 11 observaciones con valores faltantes

### Posibles Análisis Futuros
- Evolución temporal de la satisfacción
- Comparación entre centros y titulaciones
- Análisis de brechas entre percepción de alumnos y profesores
- Identificación de titulaciones con mejor/peor satisfacción